In [ ]:
import json
import csv
import pandas as pd

In [ ]:
# importing the clean results
with open('/Users/anesterov/wd/jan31/results_clean_en.json','r') as jf:
    results_en = json.load(jf)
with open('/Users/anesterov/wd/jan31/results_clean_nl.json','r') as jf:
    results_nl = json.load(jf)

### 1. Hits filter_proper_names (clean results)
count QIDs per term

In [ ]:
count_q_en = {}
for term, results in results_en.items():
    count_q_en[term] = len(set([hit['QID'] for hit in results]))

In [ ]:
count_q_nl = {}
for term, results in results_nl.items():
    count_q_nl[term] = len(set([hit['QID'] for hit in results]))

In [ ]:
with open('hits_clean.csv','w') as csv_file:
    writer = csv.writer(csv_file)
    header = ['term','lang','hits_clean']
    writer.writerow(header)
    
    for query_term, hits in count_q_en.items():
        row = [query_term,'en',hits]
        writer.writerow(row)
    for query_term, hits in count_q_nl.items():
        row = [query_term,'nl',hits]
        writer.writerow(row)

### 2. Where terms are found
query_term, wd_prefLabel, wd_aliases, wd_description

In [ ]:
# EN

wd_where_terms_found_en = {}

for term, results in results_en.items():
    pref = 0
    alias = 0
    descr = 0
    for hit in results:
        if hit['found_in'] == 'prefLabel':
            pref += 1
        if hit['found_in'] == 'aliases':
            alias += 1
        if hit['found_in'] == 'description':
            descr += 1
            
    wd_where_terms_found_en[term] = [pref,alias,descr]

In [ ]:
# NL

wd_where_terms_found_nl = {}

for term, results in results_nl.items():
    pref = 0
    alias = 0
    descr = 0
    for hit in results:
        if hit['found_in'] == 'prefLabel':
            pref += 1
        if hit['found_in'] == 'aliases':
            alias += 1
        if hit['found_in'] == 'description':
            descr += 1
            
    wd_where_terms_found_nl[term] = [pref,alias,descr]

In [ ]:
# importing query terms to get lemmas
with open('/Users/anesterov/reps/LODlit/query_terms.json','r') as jf:
    query_terms = json.load(jf)

In [ ]:
# exporting a csv of where terms are found

with open('wd_where_terms_found.csv','w') as csv_file:
    writer = csv.writer(csv_file)
    header = ['lemma','term','lang','wd_prefLabel','wd_aliases','wd_description','total']
    writer.writerow(header)
    
    for term, stats in wd_where_terms_found_en.items():
        for l, wordforms in query_terms['en'].items():
            if term in wordforms:
                lemma = l
        row = [lemma, term, 'en', stats[0], stats[1], stats[2], sum(stats)]
        writer.writerow(row)
        
    for term, stats in wd_where_terms_found_nl.items():
        for l, wordforms in query_terms['nl'].items():
            if term in wordforms:
                lemma = l
        row = [lemma, term, 'nl', stats[0], stats[1], stats[2], sum(stats)]
        writer.writerow(row)

In [ ]:
# grouping N hits by lemmas
df = pd.read_csv('wd_where_terms_found.csv')
en_df = df.loc[df['lang'] == 'en']
nl_df = df.loc[df['lang'] == 'nl']

In [ ]:
# exporting a csv with stats by lemma

with open('wd_stats_lemma.csv','w') as csv_file:
    writer = csv.writer(csv_file)
    header = ['lemma','lang','wd_pref_lemma','wd_aliases_lemma','wd_desc_lemma','total_lemma']
    writer.writerow(header)

    for group in en_df.groupby('lemma'):
        
        row = [group[0],'en',sum(group[1]['wd_prefLabel']),sum(group[1]['wd_aliases']),\
              sum(group[1]['wd_description']),sum(group[1]['total'])]
        
        writer.writerow(row)
        
    for group in nl_df.groupby('lemma'):
        
        row = [group[0],'nl',sum(group[1]['wd_prefLabel']),sum(group[1]['wd_aliases']),\
              sum(group[1]['wd_description']),sum(group[1]['total'])]
        
        writer.writerow(row)